In [9]:
#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv
import datetime
from bs4 import BeautifulSoup
import requests
import time

from collections import Counter

#Twitter API credentials
consumer_key = "1idCovKXB8ohdRRXtd64jRIxJ"
consumer_secret = "OqNSyVWG2U6jFiN3VzqeQQ2F7u7QrtC0ouSY8OHhNzs0S6JwJP"
access_key = "206260986-EsngiVAeiRLA9OKuYb7DRS0hpVLHCPvlNVLWwy7d"
access_secret = "y7RPDd3Pz9MsLwnqssvlRkYqXtU2xylR2nnCD5F3guvYe"

ModuleNotFoundError: No module named 'bs4'

In [109]:
# code from https://gist.github.com/yanofsky/5436496

def get_all_tweets(screen_name):
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		print("getting tweets before %s" % (oldest))
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		print("...%s tweets downloaded so far" % (len(alltweets)))
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.retweet_count, tweet.favorite_count] for tweet in alltweets]
	
	#write the csv	
	with open('%s_tweets.csv' % screen_name, 'w') as f:
		writer = csv.writer(f)
		writer.writerow(["id","created_at","text", "RT", "fav"])
		writer.writerows(outtweets)
	
	pass

In [110]:
if __name__ == '__main__':
	#pass in the username of the account you want to download
	get_all_tweets("jasoninthehouse")

getting tweets before 835723037433446399
...400 tweets downloaded so far
getting tweets before 803675892387983360
...600 tweets downloaded so far
getting tweets before 769307103857029119
...799 tweets downloaded so far
getting tweets before 740222520352210943
...999 tweets downloaded so far
getting tweets before 714582426115158015
...1199 tweets downloaded so far
getting tweets before 691635845703413759
...1399 tweets downloaded so far
getting tweets before 669566901832450047
...1599 tweets downloaded so far
getting tweets before 642941779642159103
...1798 tweets downloaded so far
getting tweets before 620226797007867907
...1998 tweets downloaded so far
getting tweets before 594247398869901311
...2198 tweets downloaded so far
getting tweets before 570423171880497152
...2398 tweets downloaded so far
getting tweets before 547836488344698879
...2598 tweets downloaded so far
getting tweets before 514164386697469951
...2798 tweets downloaded so far
getting tweets before 489469164017704959
.

In [231]:
# # get addtl tweets

# screen_name = "jasoninthehouse"

# #authorize twitter, initialize tweepy
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_key, access_secret)
# api = tweepy.API(auth)

# #initialize a list to hold all the tweepy Tweets
# alltweets = []

# #make initial request for most recent tweets (200 is the maximum allowed count)
# new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=str(int(all_tweets[-1]["id"]) -1))

# print(str(int(all_tweets[-1]["id"]) -1))

# #save most recent tweets
# alltweets.extend(new_tweets)

# #save the id of the oldest tweet less one
# oldest = alltweets[-1].id - 1

# #keep grabbing tweets until there are no tweets left to grab
# while len(new_tweets) > 0:
#     print("getting tweets before %s" % (oldest))

#     #all subsiquent requests use the max_id param to prevent duplicates
#     new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

#     #save most recent tweets
#     alltweets.extend(new_tweets)

#     #update the id of the oldest tweet less one
#     oldest = alltweets[-1].id - 1
#     print("...%s tweets downloaded so far" % (len(alltweets)))
    
# #transform the tweepy tweets into a 2D array that will populate the csv	
# outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.retweet_count, tweet.favorite_count] for tweet in alltweets]

# #write the csv	
# with open('chaffetz_tweets_'+str(oldest)+'.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(["id","created_at","text", "RT", "fav"])
#     writer.writerows(outtweets)

425470374638145535


IndexError: list index out of range

In [225]:
# print(all_tweets[-1]["id"])
# # all_tweets.append(alltweets)

425470374638145536


In [4]:
# open csv file as list of dicts

with open('jasoninthehouse_tweets.csv') as f:
    all_tweets = [{k: v for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

In [7]:
# make changes to tweet data

for twt in all_tweets:
    time_minus_four = (datetime.datetime.strptime(twt["created_at"], "%Y-%m-%d %H:%M:%S")- datetime.timedelta(hours=4))
    twt["new_date"] = str(time_minus_four.date())
    twt["new_time"] = str(time_minus_four.time())
    twt["hour"] = twt["new_time"][0:2]
    twt["weekday"] = time_minus_four.weekday()
    if twt["weekday"] is 7:
        twt["weekday"] = 0
    twt["weekday"] = twt["weekday"] + 1
    twt["text"] = twt["text"].replace("b'","")
    twt["text"] = twt["text"].replace('b"',"")

In [8]:
# write csv from new tweet data

out_file_name = "jasoninthehouse_tweets.csv"

with open(out_file_name, "w") as csvfile:	
	fieldnames = all_tweets[0].keys()
	writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
	writer.writeheader()
	for el in all_tweets:
		writer.writerow(el)

In [9]:
# get top 10 RT'd tweets

most_rts =  sorted(all_tweets, key=lambda k: int(k['RT']), reverse=True)[0:10]
most_rts

[{'RT': '41331',
  'created_at': '2017-06-15 02:00:34',
  'fav': '0',
  'hour': '22',
  'id': '875171141605785600',
  'new_date': '2017-06-14',
  'new_time': '22:00:34',
  'text': 'RT @realDonaldTrump: Just left hospital. Rep. Steve Scalise, one of the truly great people, is in very tough shape - but he is a real fight…',
  'weekday': 3},
 {'RT': '21835',
  'created_at': '2016-10-28 16:57:17',
  'fav': '29692',
  'hour': '12',
  'id': '792047597040971776',
  'new_date': '2016-10-28',
  'new_time': '12:57:17',
  'text': 'FBI Dir just informed me, "The FBI has learned of the existence of emails that appear to be pertinent to the investigation." Case reopened',
  'weekday': 5},
 {'RT': '11545',
  'created_at': '2016-11-06 20:15:36',
  'fav': '10761',
  'hour': '16',
  'id': '795358997083586560',
  'new_date': '2016-11-06',
  'new_time': '16:15:36',
  'text': 'FBI Dir just informed us "Based on our review, we have not changed our conclusions that we expressed in July with respect to Sec Cl

In [10]:
# get top 10 fav'd tweets

most_favs =  sorted(all_tweets, key=lambda k: int(k['fav']), reverse=True)[0:10]
most_favs

[{'RT': '21835',
  'created_at': '2016-10-28 16:57:17',
  'fav': '29692',
  'hour': '12',
  'id': '792047597040971776',
  'new_date': '2016-10-28',
  'new_time': '12:57:17',
  'text': 'FBI Dir just informed me, "The FBI has learned of the existence of emails that appear to be pertinent to the investigation." Case reopened',
  'weekday': 5},
 {'RT': '6162',
  'created_at': '2017-05-16 22:58:45',
  'fav': '14286',
  'hour': '18',
  'id': '864616135466950656',
  'new_date': '2017-05-16',
  'new_time': '18:58:45',
  'text': '.@GOPoversight is going to get the Comey memo, if it exists. I need to see it sooner rather than later. I have my subpoena pen ready.',
  'weekday': 2},
 {'RT': '11545',
  'created_at': '2016-11-06 20:15:36',
  'fav': '10761',
  'hour': '16',
  'id': '795358997083586560',
  'new_date': '2016-11-06',
  'new_time': '16:15:36',
  'text': 'FBI Dir just informed us "Based on our review, we have not changed our conclusions that we expressed in July with respect to Sec Clinto

In [18]:
# get top 100 word freq count

all_words = ""
for twt in all_tweets:
    all_words = all_words + " " + twt["text"]
    
words_freq =Counter(all_words.lower().replace(":","").split()).most_common()
    
most_used_words = Counter(all_words.lower().split()).most_common()[0:100]

In [51]:
# get most used hashtags

hashtags_freq = []

most_used_hashtags = []

for word in words_freq:
    if word[0][0] == "#":
        hashtags_freq.append(word)
        if word[1] > 3:
            most_used_hashtags.append(word)
            
# fix utah

for ind, ht in enumerate(most_used_hashtags):
    if ht[0] == "#utah…":
        add_utah_num = ht[1]
        most_used_hashtags.pop(ind)
        

for ind,ht in enumerate(most_used_hashtags):
    if ht[0] == "#utah":
        most_used_hashtags[ind] = ["#utah", ht[1]+add_utah_num]
  
for ht in most_used_hashtags:
    print(ht[0] + " - " + str(ht[1]))

#benghazi - 38
#tbt - 35
#utah - 34
#utpol - 29
#irs - 26
#tcot - 20
#fastandfurious - 14
#ut3rddistrict - 14
#irsfail - 11
#usa - 9
#secretservice - 8
#neverforget - 7
#breaking - 7
#emerycounty - 7
#nomidnightmonument - 6
#epa - 6
#nature - 5
#july4 - 5
#americanforkcanyon - 4
#taxday - 4
#oneluckyguy - 4
#plannedparenthood - 4
#byufootball - 4
#gopoversight - 4
#natgeo - 4
#sxsw - 4


In [36]:
# get most used hashtags

ats_freq = []

most_used_ats = []

for word in words_freq:
    if word[0][0] == "@" and word[0] != "@":
        ats_freq.append(word)
        if word[1] > 14:
            most_used_ats.append(word)
  
for at in most_used_ats:
    print(at[0] + " - " + str(at[1]))

@gopoversight - 505
@foxnews - 165
@youtube - 51
@jasoninthehouse - 49
@politico - 41
@cnn - 37
@kslnewsradio - 37
@sltrib - 31
@speakerryan - 30
@usatoday - 27
@wsj - 25
@washingtonpost - 24
@greta - 23
@abc - 22
@kslcom - 21
@dailycaller - 21
@dougwrightshow - 18
@tgowdysc - 16
@wolfblitzer - 15
@speakerboehner - 15


In [209]:
# scrape twitter vitals for chaffetz' class in House


member_uns = ["JasonInTheHouse",
"RepMikeCoffman",
"RepGuthrie",
"GreggHarper",
"Rep_Hunter",
"RepLynnJenkins",
"RepLanceNJ7",
"RepBlainePress",
"RepMcClintock",
"RepPeteOlson",
"RepErikPaulsen",
"CongBillPosey",
"DrPhilRoe",
"RepRooney",
"CongressmanGT"]

class_data = []

url_prefix = "http://www.twitter.com/"

for un in member_uns:
    resp = requests.get(url_prefix + un)
    html = resp.text
    soup = BeautifulSoup(html,"html.parser")
    vitals = soup.find_all("a", {"class":"ProfileNav-stat"})
    d = {}
    for vital in vitals:
        d[vital.find_all("span")[0].getText()] = vital.find_all("span")[2].getText()  
    d["un"] = un
    class_data.append(d)
    print("finished " + un)
    time.sleep(2) #seconds
    
longest_vital = [0,0]    

for ind, member in enumerate(class_data):
    member["Tweets"] = member["Tweets"].replace("\n","").strip()
    if len(member.keys()) > longest_vital[1]:
        longest_vital[0] = ind
        longest_vital[1] = len(member.keys())
    
    
out_file_name = "class_09_twitter_vitals.csv"

with open(out_file_name, "w") as csvfile:
    fieldnames = class_data[longest_vital[0]].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for el in class_data:
        writer.writerow(el)

finished JasonInTheHouse
finished RepMikeCoffman
finished RepGuthrie
finished GreggHarper
finished Rep_Hunter
finished RepLynnJenkins
finished RepLanceNJ7
finished RepBlainePress
finished RepMcClintock
finished RepPeteOlson
finished RepErikPaulsen
finished CongBillPosey
finished DrPhilRoe
finished RepRooney
finished CongressmanGT


In [22]:
all_tweets[3]

{'RT': '292',
 'created_at': '2017-06-25 20:05:19',
 'fav': '966',
 'hour': '16',
 'id': '879068006466428934',
 'new_date': '2017-06-25',
 'new_time': '16:05:19',
 'text': 'Rep. Chaffetz: Congress needs to be watchdog over Mueller https://t.co/4tJoXpPPPs',
 'weekday': 7}

In [55]:
member_uns = ["RepAmata"], ["SenToddYoung"], ["SenJoniErnst"], ["senthomtillis"], ["RepKevinYoder"], ["RepJohnYarmuth"], ["RepDonYoung"], ["Rep_SteveWomack"], ["RepWilson"], ["RepWebster"], ["MarkWarner"], ["RobWittman"], ["SenWhitehouse"], ["PeterWelch"], ["RepWalberg"], ["RepDWStweets"], ["RepJoeWilson"], ["RepGregWalden"], ["RonWyden"], ["SenatorWicker"], ["MaxineWaters"], ["ChrisVanHollen"], ["RepVisclosky"], ["NydiaVelazquez"], ["SenatorTomUdall"], ["RepFredUpton"], ["RepTipton"], ["RepPaulTonko"], ["RepDinaTitus"], ["CongressmanGT"], ["NikiInTheHouse"], ["SenatorTester"], ["RepMikeTurner"], ["pattiberi"], ["SenToomey"], ["RepThompson"], ["SenJohnThune"], ["MacTXPress"], ["BennieGThompson"], ["AustinScottGA08"], ["RepSteveStivers"], ["RepTerriSewell"], ["SenatorTimScott"], ["RepDavid"], ["SenatorShaheen"], ["RepSchrader"], ["SteveScalise"], ["RepSpeier"], ["RepAdrianSmith"], ["RepSires"], ["RepDavidScott"], ["RepLindaSanchez"], ["RepBillShuster"], ["RepAdamSchiff"], ["CongMikeSimpson"], ["JanSchakowsky"], ["SenStabenow"], ["LamarSmithTX21"], ["RepAdamSmith"], ["LouiseSlaughter"], ["RepShimkus"], ["BradSherman"], ["SenShelby"], ["PeteSessions"], ["RepJoseSerrano"], ["JimPressOffice"], ["BobbyScott"], ["SenSchumer"], ["SenSanders"], ["SenRubioPress"], ["RepDennisRoss"], ["ToddRokita"], ["RepMarthaRoby"], ["RepRichmond"], ["RepJimRenacci"], ["RepTomReed"], ["TomRooney"], ["DrPhilRoe"], ["PeterRoskam"], ["DaveReichert"], ["RepTimRyan"], ["Call_Me_Dutch"], ["RepMikeRogersAL"], ["SpeakerRyan"], ["RepBobbyRush"], ["RepEdRoyce"], ["RepRoybalAllard"], ["RosLehtinen"], ["RepHalRogers"], ["SenPatRoberts"], ["SenJackReed"], ["RepMikeQuigley"], ["CongPalazzo"], ["CongBillPosey"], ["RepJaredPolis"], ["ChelliePingree"], ["SenGaryPeters"], ["RepErikPaulsen"], ["RepPerlmutter"], ["JudgeTedPoe"], ["RepStevePearce"], ["RepDavidEPrice"], ["SenRobPortman"], ["NancyPelosi"], ["BillPascrell"], ["FrankPallone"], ["RepPeteOlson"], ["RepKristiNoem"], ["Rep_DevinNunes"], ["GraceNapolitano"], ["EleanorNorton"], ["SenBillNelson"], ["RepRichardNeal"], ["RepJerryNadler"], ["Sen_JoeManchin"], ["RepMeehan"], ["RepMcKinley"], ["RepTomMarino"], ["RepMcClintock"], ["SenJeffMerkley"], ["McCaskillOffice"], ["RepMcNerney"], ["GOPLeader"], ["DorisMatsui"], ["RepGwenMoore"], ["CathyMcMorris"], ["RepKenMarchant"], ["RepMcCaul"], ["PatrickMcHenry"], ["LisaMurkowski"], ["RepTimMurphy"], ["BettyMcCollum04"], ["GregoryMeeks"], ["PattyMurray"], ["JerryMoran"], ["SenatorMenendez"], ["McConnellPress"], ["RepMcGovern"], ["SenJohnMcCain"], ["SenMarkey"], ["RepMaloney"], ["SenMikeLee"], ["USRepLong"], ["SenatorLankford"], ["Raul_Labrador"], ["RepBenRayLujan"], ["RepBlainePress"], ["RepLanceNJ7"], ["BobLatta"], ["DaveLoebsack"], ["RepDLamborn"], ["RepLipinski"], ["RepStephenLynch"], ["RepRickLarsen"], ["JimLangevin"], ["RepJohnLarson"], ["RepLoBiondo"], ["RepBarbaraLee"], ["RepFrankLucas"], ["NitaLowey"], ["RepZoeLofgren"], ["RepJohnLewis"], ["RepSandyLevin"], ["SenatorLeahy"], ["RepKinzinger"], ["MikeKellyPA"], ["USRepKeating"], ["SteveKingIA"], ["RepPeteKing"], ["RepRonKind"], ["RepMarcyKaptur"], ["SenRonJohnson"], ["RepBillJohnson"], ["RepLynnJenkins"], ["Jim_Jordan"], ["RepHankJohnson"], ["RepWalterJones"], ["SamsPressShop"], ["RepEBJ"], ["JacksonLeeTX18"], ["DarrellIssa"], ["InhofePress"], ["SenJohnHoeven"], ["RepHultgren"], ["RepHuizenga"], ["HerreraBeutler"], ["RepHartzler"], ["Rep_Hunter"], ["JAHimes"], ["MartinHeinrich"], ["GreggHarper"], ["MazieHirono"], ["SenDeanHeller"], ["RepBrianHiggins"], ["RepHensarling"], ["WhipHoyer"], ["SenOrrinHatch"], ["RepHastingsFL"], ["RepMGriffith"], ["TGowdySC"], ["RepGosar"], ["RepBobGibbs"], ["SenCoryGardner"], ["RepTomGraves"], ["RepGaramendi"], ["RepAlGreen"], ["RepLouieGohmert"], ["RepraulGrijalva"], ["RepGutierrez"], ["RepGeneGreen"], ["ChuckGrassley"], ["RepKayGranger"], ["GrahamBlog"], ["RepGoodlatte"], ["RepBillFlores"], ["Farenthold"], ["RepChuck"], ["RepMarciaFudge"], ["VirginiaFoxx"], ["JeffFortenberry"], ["RepTrentFranks"], ["JeffFlake"], ["SenFeinstein"], ["KeithEllison"], ["SenatorEnzi"], ["RepAnnaEshoo"], ["RepEliotEngel"], ["DesJarlaisTN04"], ["RepJeffDuncan"], ["RepSeanDuffy"], ["RepJeffDenham"], ["RepTedDeutch"], ["SenDonnelly"], ["RepCharlieDent"], ["MarioDB"], ["SenatorDurbin"], ["RepJohnDuncanJr"], ["USRepMikeDoyle"], ["RepLloydDoggett"], ["RosaDeLauro"], ["RepDianaDeGette"], ["RepPeterDeFazio"], ["SenCoonsOffice"], ["RepRickCrawford"], ["RepCicilline"], ["RepJudyChu"], ["GerryConnolly"], ["RepMikeCoffman"], ["JasonInTheHouse"], ["RepAndreCarson"], ["SenBobCorker"], ["SenBobCasey"], ["RepJoeCourtney"], ["RepCohen"], ["RepYvetteClarke"], ["RepCuellar"], ["ConawayTX11"], ["RepCleaver"], ["RepJimCosta"], ["JohnCornyn"], ["TomColeOK04"], ["JudgeCarter"], ["CongCulberson"], ["SenCapito"], ["RepJoeCrowley"], ["RepMikeCapuano"], ["SenatorCollins"], ["RepCummings"], ["MikeCrapo"], ["RepJimCooper"], ["RepJohnConyers"], ["Clyburn"], ["RepSteveChabot"], ["SenatorCarper"], ["SenatorCardin"], ["SenatorCantwell"], ["KenCalvert"], ["SenBlumenthal"], ["RepLarryBucshon"], ["RepMoBrooks"], ["RepDianeBlack"], ["RepKarenBass"], ["RepLouBarletta"], ["SenBennetCo"], ["SenJohnBarrasso"], ["VernBuchanan"], ["RepGusBilirakis"], ["GKButterfield"], ["RepRobBishop"], ["MichaelCBurgess"], ["MarshaBlackburn"], ["JohnBoozman"], ["RepBrady"], ["SenatorBurr"], ["SenSherrodBrown"], ["RepKevinBrady"], ["RoyBlunt"], ["BlumenauerMedia"], ["SanfordBishop"], ["RepJoeBarton"], ["SenAlexander"], ["Robert_Aderholt"], ["RepDelBene"], ["RepBonamici"], ["SenThadCochran"], ["SenAngusKing"], ["MarkAmodeiNV2"], ["RepLujanGrisham"], ["RepDeSantis"], ["RepJohnDelaney"], ["RepSwalwell"], ["RepMullin"], ["RepJoeKennedy"], ["SusanWBrooks"], ["RepWalorski"], ["RepTomRice"], ["RepGraceMeng"], ["RepLukeMesser"], ["RepSeanMaloney"], ["RepDennyHeck"], ["RepHolding"], ["USRepRickNolan"], ["RepMarkTakano"], ["RepJeffries"], ["RepDaveJoyce"], ["RepLowenthal"], ["RepDougCollins"], ["TXRandy14"], ["RepDerekKilmer"], ["TulsiPress"], ["SteveDaines"], ["RepTedYoho"], ["RepEsty"], ["RepAnnieKuster"], ["RepDanKildee"], ["RepCartwright"], ["SenTomCotton"], ["RepCheri"], ["RepBera"], ["RepBillFoster"], ["RepPittenger"], ["RepMarkMeadows"], ["RepLoisFrankel"], ["RepRichHudson"], ["RepVeasey"], ["RepFilemonVela"], ["RepScottPerry"], ["CongressmanRuiz"], ["RepPaulCook"], ["RepKevinCramer"], ["RepChrisCollins"], ["RepAnnWagner"], ["SenWarren"], ["RodneyDavis"], ["RepChrisStewart"], ["RepAndyBarr"], ["SenTedCruz"], ["RepDannyDavis"], ["RepSamGraves"], ["RepJBridenstine"], ["SenatorBaldwin"], ["RepRobWoodall"], ["RepRWilliams"], ["RepSusanDavis"], ["SenDuckworth"], ["RepBetoORourke"], ["SenatorFischer"], ["RepHuffman"], ["SenatorHeitkamp"], ["RepAndyHarrisMD"], ["RepDonaldPayne"], ["RepScottPeters"], ["KeithRothfus"], ["JoaquinCastrotx"], ["RepBradWenstrup"], ["RepDavidValadao"], ["RepSinema"], ["USRepRodney"], ["SenFranken"], ["RepGuthrie"], ["RepLaMalfa"], ["USRepKCastor"], ["SenatorIsakson"], ["RepCardenas"], ["RepTimWalz"], ["RepMarkPocan"], ["RepJuanVargas"], ["SenKaineOffice"], ["SenBrianSchatz"], ["RepSarbanes"], ["JuliaBrownley26"], ["RepThomasMassie"], ["RepRobinKelly"], ["SenGillibrand"], ["SenatorRisch"], ["RepBeatty"], ["RepSanfordSC"], ["RepJasonSmith"], ["RepChrisSmith"], ["senmurphyoffice"], ["senbookeroffice"], ["RepKClark"], ["RepByrne"], ["RepNewhouse"], ["RepEvanJenkins"], ["RepRatcliffe"], ["RepLoudermilk"], ["RepBuddyCarter"], ["RepCurbelo"], ["RepTomMacArthur"], ["RepBost"], ["RepAdams"], ["RepComstock"], ["RepPoliquin"], ["RepMoolenaar"], ["RepStefanik"], ["RepRyanCostello"], ["RepDavidRouzer"], ["USRepGaryPalmer"], ["RepMcSally"], ["RepTomEmmer"], ["RepRussell"], ["RepAbraham"], ["RepDavidYoung"], ["RepLeeZeldin"], ["RepJohnKatko"], ["RepWesterman"], ["RepFrenchHill"], ["RepBrianBabin"], ["RepAlexMooney"], ["repdavetrott"], ["RepKenBuck"], ["RepRodBlum"], ["RepMikeBishop"], ["RepKathleenRice"], ["RepDebDingell"], ["NormaJTorres"], ["SenSasse"], ["SteveKnight25"], ["SenDanSullivan"], ["repmialove"], ["reprickallen"], ["SenatorRounds"], ["RepMimiWalters"], ["CongBoyle"], ["congressmanhice"], ["reppeteaguilar"], ["RepRubenGallego"], ["sendavidperdue"], ["staceyplaskett"], ["RepBonnie"], ["hurdonthehill"], ["RepTedLieu"], ["RepGarretGraves"], ["RepDeSaulnier"], ["RepMarkWalker"], ["teammoulton"], ["repdonbeyer"], ["RepGrothman"], ["DonaldNorcross"], ["RepLawrence"], ["RepDaveBrat"], ["reptrentkelly"], ["reprohrabacher"], ["RepDanDonovan"], ["RepLaHood"], ["RepJimBanks"], ["RepValDemings"], ["RepBRochester"], ["repraskin"], ["RepJimmyPanetta"], ["repcharliecrist"], ["drnealdunnfl2"], ["RepBarragan"], ["RepAndyBiggsAZ"], ["repbrianmast"], ["RepMcEachin"], ["WarrenDavidson"], ["RepDrewFerguson"], ["RepGonzalez"], ["RepJoshG"], ["Rep_Tom_Garrett"], ["sencortezmasto"], ["RepRoKhanna"], ["repsheaporter"], ["repjenniffer"], ["RepStephMurphy"], ["reptrey"], ["RepArrington"], ["RepLizCheney"], ["RepCarbajal"], ["RepTomSuozzi"], ["repschneider"], ["RepJohnFaso"], ["RepTenney"], ["RepTedBudd"], ["RepJayapal"], ["repbrianfitz"], ["repohalleran"], ["RepRooney"], ["RepKihuen"], ["SenJohnKennedy"], ["RepAlLawsonJr"], ["RepPaulMitchell"], ["congressmanraja"], ["Senatorhassan"], ["RepEspaillat"], ["repdavidkustoff"], ["RepJasonLewis"], ["RepMattGaetz"], ["RepDarrenSoto"], ["RepJackBergman"], ["RepGallagher"], ["RepMarshall"], ["RepDwightEvans"], ["RepRutherfordFL"], ["RepMikeJohnson"], ["repjackyrosen"], ["reploucorrea"], ["repdonbacon"], ["RepSmucker"], ["SenatorStrange"], ["RepHanabusa"], ["SenKamalaHarris"], ["RepClayHiggins"], ["RepAnthonyBrown"], ["RepScottTaylor"], ["RepRonEstes"]

member_data = []

url_prefix = "http://www.twitter.com/"

for un in member_uns:
    resp = requests.get(url_prefix)
    html = resp.text
    soup = BeautifulSoup(html,"html.parser")
    vitals = soup.find_all("a", {"class":"ProfileNav-stat"})
    d = {}
    for vital in vitals:
        d[vital.find_all("span")[0].getText()] = vital.find_all("span")[2].getText()  
    d["un"] = un
    member_data.append(d)
    print("finished " + un)
    time.sleep(1) #seconds
    
longest_vital = [0,0]    

for ind, member in enumerate(member_data):
    member["Tweets"] = member["Tweets"].replace("\n","").strip()
    if len(member.keys()) > longest_vital[1]:
        longest_vital[0] = ind
        longest_vital[1] = len(member.keys())
    
    
out_file_name = "all_twitter_vitals.csv"

with open(out_file_name, "w") as csvfile:
    fieldnames = member_data[longest_vital[0]].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for el in member_data:
        writer.writerow(el)

NameError: name 'BeautifulSoup' is not defined

In [10]:
sys.executable

NameError: name 'sys' is not defined